# 사고 파일 불러오기

연도는 이미 열에 존재하기 때문에 모든 데이터 병합

In [44]:
import pandas as pd

file_path = '../data/Incident.xlsx'
sheet_names = [str(year) for year in range(2020,2025)]

dfs = [pd.read_excel(file_path, sheet_name = sheet) for sheet in sheet_names]

merged_df = pd.concat(dfs, ignore_index = True)

In [45]:
merged_df

,구분,지역,교육청,학교급,사고자구분,사고자학년,사고자성별,사고발생일,사고발생시각,사고발생요일,사고시간,사고장소,사고부위,사고형태,사고당시활동
0,A0000001,경북,경상북도교육청,고등학교,일반학생,3학년,남,2017-03-27,12:50,월,식사시간(간식 포함),일반(교과)교실,치아,고정된 물체와의 부딪힘,리듬·기계체조
1,A0000002,서울,동부교육지원청,초등학교,일반학생,5학년,여,2017-05-24,13:20,수,체육,강당(체육관),복합부위,그밖의 손상 사고,기타
2,A0000003,대구,남부교육지원청,초등학교,일반학생,2학년,남,2017-09-27,11:00,수,쉬는시간,화장실,이마,고정된 물체와의 부딪힘,"장난, 놀이"
3,A0000004,광주,동부교육지원청,초등학교,일반학생,3학년,여,2017-12-22,11:50,금,현장학습,기타 교외,치아,넘어짐,"걷기/뛰기, 오르내리기"
4,A0000005,서울,강남서초교육지원청,초등학교,일반학생,1학년,남,2018-02-08,12:10,목,식사시간(간식 포함),복도,입술 및 구강,고정된 물체와의 부딪힘,"장난, 놀이"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689248,E0211646,서울,서부교육지원청,중학교,일반학생,2학년,여,2024-12-31,14:05,화,그 밖의 교육활동 시간,운동장,복합부위,움직이는 물체와의 부딪힘,기타 구기
689249,E0211647,광주,서부교육지원청,중학교,일반학생,1학년,남,2024-12-31,09:50,화,쉬는시간,강당(체육관),손가락,움직이는 물체와의 부딪힘,휴식
689250,E0211648,서울,남부교육지원청,초등학교,일반학생,6학년,여,2024-12-20,10:20,금,체육,강당(체육관),손가락,움직이는 물체와의 부딪힘,피구
689251,E0211649,서울,서울특별시교육청,고등학교,일반학생,2학년,남,2024-12-30,16:45,월,방과후과정,특별교실(과학실 외),손가락,"베임, 절단",일반실습


# 전처리

- na행 제거
  - 존재하는 na는 교직원의 사고로 학교안전사고의 대부분인 학생 위주 분석하기 위함
- 불필요 feature 제거
  - 안전교육홍보 목적: 일반화를 위해 지역을 남기고 교육청은 제거
  - 사고발생일에서 월만 남김: 년도와 일에 비해 월은 방학여부, 계절 정보를 담고 있음
  - 사고발생 시각의 분석을 위해 ':' 제거
- 파생변수 생성
  - 학교급과 학년을 조합하여 일반적인 나이 열을 생성: 발달적인 부분을 보기 위함

In [46]:
merged_df.isna().sum()

구분           0
지역           0
교육청          0
학교급          0
사고자구분        0
사고자학년     1632
사고자성별        0
사고발생일        0
사고발생시각       0
사고발생요일       0
사고시간         0
사고장소         0
사고부위         0
사고형태         0
사고당시활동       0
dtype: int64

In [47]:
# na 드롭
merged_df.dropna(inplace=True)

# 사고발생월 생성
merged_df['사고발생월'] = pd.to_datetime(merged_df['사고발생일']).dt.month

# 필요없는 열 제거
merged_df.drop(columns=['교육청'], inplace=True)
merged_df.drop(columns=['사고발생일'], inplace=True)
merged_df.drop(columns=['구분'], inplace = True)

In [53]:
def age(x):
    a = 0
    if x['학교급'] == '초등학교':
        a = 7
    elif x['학교급'] == '중학교':
        a = 13
    else:
        a = 16
    return a

In [55]:
def age2(y):
    b = 0
    if y['사고자학년'] == '1학년':
        b = y['추정나이'] + 0
    elif y['사고자학년'] == '2학년':
        b = y['추정나이'] + 1
    elif y['사고자학년'] == '3학년':
        b = y['추정나이'] + 2
    elif y['사고자학년'] == '4학년':
        b = y['추정나이'] + 3
    elif y['사고자학년'] == '5학년':
        b = y['추정나이'] + 4
    elif y['사고자학년'] == '6학년':
        b = y['추정나이'] + 5
    return b

In [63]:
def times(z):
    c = 0
    if ':' in z['사고발생시각']:
        c = z['사고발생시각'].replace(':', '')
    return int(c)

In [66]:
merged_df['추정나이'] = merged_df.apply(age, axis=1)
merged_df['추정나이'] = merged_df.apply(age2, axis=1)
merged_df['사고발생시각'] = merged_df.apply(times, axis=1)

In [69]:
merged_df

,지역,학교급,사고자구분,사고자학년,사고자성별,사고발생시각,사고발생요일,사고시간,사고장소,사고부위,사고형태,사고당시활동,사고발생월,추정나이
0,경북,고등학교,일반학생,3학년,남,1250,월,식사시간(간식 포함),일반(교과)교실,치아,고정된 물체와의 부딪힘,리듬·기계체조,3,18
1,서울,초등학교,일반학생,5학년,여,1320,수,체육,강당(체육관),복합부위,그밖의 손상 사고,기타,5,11
2,대구,초등학교,일반학생,2학년,남,1100,수,쉬는시간,화장실,이마,고정된 물체와의 부딪힘,"장난, 놀이",9,8
3,광주,초등학교,일반학생,3학년,여,1150,금,현장학습,기타 교외,치아,넘어짐,"걷기/뛰기, 오르내리기",12,9
4,서울,초등학교,일반학생,1학년,남,1210,목,식사시간(간식 포함),복도,입술 및 구강,고정된 물체와의 부딪힘,"장난, 놀이",2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689248,서울,중학교,일반학생,2학년,여,1405,화,그 밖의 교육활동 시간,운동장,복합부위,움직이는 물체와의 부딪힘,기타 구기,12,14
689249,광주,중학교,일반학생,1학년,남,950,화,쉬는시간,강당(체육관),손가락,움직이는 물체와의 부딪힘,휴식,12,13
689250,서울,초등학교,일반학생,6학년,여,1020,금,체육,강당(체육관),손가락,움직이는 물체와의 부딪힘,피구,12,12
689251,서울,고등학교,일반학생,2학년,남,1645,월,방과후과정,특별교실(과학실 외),손가락,"베임, 절단",일반실습,12,17


In [68]:
merged_df.to_csv('../data/Incident_pre.csv')